In [1]:
!pip install jupyter_dash

In [1]:
!pip3 install pandas dash

In [3]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

--2021-10-25 07:44:49--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3033 (3.0K) [text/csv]
Saving to: ‘spacex_launch_dash.csv.1’

spacex_launch_dash. 100%[===================>]   2.96K  --.-KB/s    in 0s      

2021-10-25 07:44:49 (78.3 MB/s) - ‘spacex_launch_dash.csv.1’ saved [3033/3033]



In [2]:
# Import required libraries
from jupyter_dash import JupyterDash
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

<ipython-input-2-7a825774df48>:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-2-7a825774df48>:6: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [4]:
#When running in JupyterHub or Binder, call the infer_jupyter_config function to detect the proxy configuration.
JupyterDash.infer_jupyter_proxy_config()

OSError: Unable to communicate with the jupyter_dash notebook or JupyterLab 
extension required to infer Jupyter configuration.

In [5]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

In [6]:
spacex_df['Launch Site'].unique().tolist()

['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40']

In [ ]:
sites=spacex_df['Launch Site'].unique().tolist()
sites.insert(0,'All Sites')

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                                options=[
                                                {'label': I, 'value': I} for I in sites],
                                                value='All Sites',
                                                placeholder="Select a Launch Site here",
                                                searchable=True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id = 'payload-slider',
                                                min = 0, max = 10000, step = 1000,
                                                marks = {0: '0 kg',
                                                        2500: '2500',
                                                        5000: '5000',
                                                        7500: '7500',
                                                        10000: '10000'},
                                                value = [min_payload, max_payload]
                                                        ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output

@app.callback(Output(component_id='success-pie-chart',component_property='figure'),
                Input(component_id='site-dropdown',component_property='value'))
def get_pie(value):
    filtered_df = spacex_df
    if value == 'All Sites':
        fig = px.pie(filtered_df, values='class',names='Launch Site',
        title= 'Total Success Launches By Site')
        return fig

    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == value]
        filtered_df.groupby(['Launch Site','class']).size().reset_index(name='class count')
        fig = px.pie(filtered_df, values = 'class count', names = 'class', title = "Total Success Launches for site {value}")
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output

@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
            [Input(component_id = 'site-dropdown', component_property = 'value'), 
            Input(component_id = 'payload-slider', component_property = 'value')])

def get_scatter(value1,value2):
    filtered_df2_1 = spacex_df[(spacex_df['Payload Mass (kg)'] > value2[0]) & (spacex_df['Payload Mass (kg)'] < value2[1])]

    if value1 == 'All Sites':
        fig = px.scatter(filtered_df2_1, x  = "Payload Mass (kg)", y = "class", \
	color = "Booster Version Category", \
	title= "Correlation between Payload and Success for All sites")
        return fig 
    else: 
        filtered_df2_2 = filtered_df2_1[filtered_df2_1['Launch Site']==value1]
        fig = px.scatter(filtered_df2_2, x  = 'Payload Mass (kg)', y = 'class', \
	color = 'Booster Version Category', \
	title = "Correlation between Payload and Success for site {value1}")
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
